# Training an Emotion-Aware Chatbot

This notebook demonstrates the process of fine-tuning a **DialoGPT** model to create an emotion-aware chatbot. The model is trained on the DailyDialog dataset, which contains multi-turn dialogues annotated with emotions. The goal is to enable the chatbot to generate responses that are sensitive to the emotional context of the conversation.

# Setup

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd drive/MyDrive/

/content/drive/MyDrive


In [3]:
!pip install torch
!pip install transformers
!pip install datasets
!pip install accelerate -U
!pip install nltk
!python -m nltk.downloader punkt

<frozen runpy>:128: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [2]:
import torch
import torch.nn as nn
from tqdm import tqdm
import numpy as np
import os
import json
import pandas as pd
import kagglehub

from datasets import load_dataset
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TextDataset, DataCollatorForLanguageModeling
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from transformers import BertForSequenceClassification

from transformers import Trainer, TrainingArguments
from transformers import AutoModelForCausalLM, AutoTokenizer

import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau

from transformers import SchedulerType

In [7]:
!pip install datasets==3.6.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 27.9 MB/s eta 0:00:00
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0


## Loading model and dataset and preprocessing the data
Load the dailydialog dataset and preprocess it by adding emotion tags to each utterance. The dataset is tokenized using the DialoGPT tokenizer, preparing it for training.

In [4]:
model = GPT2LMHeadModel.from_pretrained('microsoft/DialoGPT-small')
tokenizer = AutoTokenizer.from_pretrained('microsoft/DialoGPT-small', padding_side='left')
tokenizer.pad_token = tokenizer.eos_token

dataset = load_dataset("daily_dialog")
emotion_dict = {0: 'neutral', 1: 'anger', 2: 'disgust', 3: 'fear', 4: 'happiness', 5: 'sadness', 6: 'surprise'}

def encode_with_emotion(examples):
    # Add emotion tags to both the user's and the agent's utterances
    dialog_with_emotions = []
    for dialog, emotion in zip(examples['dialog'], examples['emotion']):
        dialog_with_emotion = []
        for i in range(len(dialog)):
            emotion_tag = emotion_dict[emotion[i]]
            dialog_with_emotion.append(f"{emotion_tag}: {dialog[i]}")

        dialog_with_emotions.append(" ".join(dialog_with_emotion))  # Join each dialog into a string

    encoded = tokenizer(dialog_with_emotions, truncation=True, padding='max_length', max_length=128)
    encoded['labels'] = encoded['input_ids'][:]
    return encoded

encoded_datasets = dataset.map(encode_with_emotion, batched=True)

## Fine-Tune the DialoGPT Model
Configure the training parameters and fine-tune the DialoGPT model on the preprocessed dataset.

The model is trained for 10 epochs with a cosine learning rate scheduler.

In [7]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

training_args = TrainingArguments(
    output_dir="./DialoGPT-emotion",
    overwrite_output_dir=True,
    num_train_epochs=10,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=0,
    weight_decay=0.01,
    learning_rate=5e-5,
    lr_scheduler_type=SchedulerType.COSINE,
    save_total_limit=2,
    prediction_loss_only=True,
    fp16=True,
    save_strategy="steps",
    save_steps=4000,
    eval_strategy="steps",
    eval_steps=500,
    load_best_model_at_end=True,
    metric_for_best_model="loss",
    seed=42,
    report_to=[]
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=encoded_datasets["train"],
    eval_dataset=encoded_datasets["validation"]
)

trainer.train()


Step,Training Loss,Validation Loss
500,2.512500,2.210340
1000,2.227600,2.141124
1500,2.123000,2.101796
2000,2.026200,2.072141
2500,1.946700,2.058032
3000,1.893600,2.051465
3500,1.849400,2.040915
4000,1.793300,2.031112
4500,1.759500,2.033555
5000,1.737600,2.033683


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


## Save model

In [8]:
trainer.save_model("./DialoGPT-emotion")
tokenizer.save_pretrained("./DialoGPT-emotion")

('./DialoGPT-emotion/tokenizer_config.json',
 './DialoGPT-emotion/special_tokens_map.json',
 './DialoGPT-emotion/chat_template.jinja',
 './DialoGPT-emotion/vocab.json',
 './DialoGPT-emotion/merges.txt',
 './DialoGPT-emotion/added_tokens.json',
 './DialoGPT-emotion/tokenizer.json')

## Evaluation
Evaluate the fine-tuned model on the **validation dataset** to assess its performance.

In [9]:
eval_results = trainer.evaluate()
print("Validation Results:", eval_results)

Validation Results: {'eval_loss': 2.0311121940612793, 'eval_runtime': 4.7782, 'eval_samples_per_second': 209.283, 'eval_steps_per_second': 3.349, 'epoch': 10.0}


The fine-tuned DialoGPT-small model achieved a validation loss of 2.031 after 10 epochs on the DailyDialog dataset, indicating reasonable performance in generating emotion-aware responses.

Given resource constraints, this result is acceptable, demonstrating the model's ability to learn emotional context.

Further improvements will be explored .